# Process the feature importance values from STREAMLINE outputs

In [1]:
import glob
import numpy as np
import pandas as pd
import os

# read the original feature importance values

def readFI(data_path):
    
    # give the order of the algs
    alg_13_list = np.array(['NB', 'EN', 'LR', 'DT', 'RF', 'GB', 'XGB', 'LGB', 'CGB', 'SVM', 'ANN', 'KNN', 'GP'])
    FI_mean = []
    
    #Determine file extension of datasets in target folder:
    file_count = 0
    unique_datanames = []
    for alg_name in alg_13_list:
        for dataset_path in sorted(glob.glob(data_path+'/*'), key=os.path.getmtime):
            dataset_path = str(dataset_path).replace('\\','/')    
            #print('---------------------------------------------------------------------------------')
            #print(dataset_path)
            file_extension = dataset_path.split('/')[-1].split('.')[-1]
            data_name = dataset_path.split('/')[-1].split('.')[0] #Save unique dataset names so that analysis is run only once if there is both a .txt and .csv version of dataset with same name.
            if data_name==alg_name+'_FI':
                if file_extension == 'txt' or file_extension == 'csv':
                    if data_name not in unique_datanames:
                        unique_datanames.append(data_name)
                        df = pd.read_csv(dataset_path,na_values='NA',sep=',')
                        fi_mean = []
                        for i in range(0,df.shape[1]):
                            fi_mean.append(df.iloc[:,i].mean()) # average of k-cv folds
                        fi_mean = np.array(fi_mean)
                        FI_mean.append(fi_mean)
                        file_count += 1
    FI = np.array(FI_mean)
    ROIs = df.columns.values
    ROI_name = []
    for i in range(len(ROIs)):
        ROI_name.append(ROIs[i].split('_')[1]+"_"+ROIs[i].split('_')[2])  # extract ROIs names
        
    if file_count == 0: #Check that there was at least 1 dataset
        raise Exception("There must be at least one .txt or .csv dataset in data_path directory")
    return FI,ROI_name

In [ ]:
# recaling methods

def rescaleFI(FI,ROI_name,path):
    
    algs = np.array(['NB', 'EN', 'LR', 'DT', 'RF', 'GB', 'XGB', 'LGB', 'CGB', 'SVM', 'ANN', 'KNN', 'GP','SUM'])
    
    # Normarlization

    FI_norm = np.empty((FI.shape[0],FI.shape[1]))
    for i in range(0,FI.shape[0]):
        for j in range(0,FI.shape[1]):
            if (FI[i,j]<=0):
                FI_norm[i,j]=0
            else:
                FI_norm[i,j] = FI[i,j]/np.max(FI[i,:])

    df =  pd.DataFrame(FI)
    df_n =  pd.DataFrame(FI_norm)

    # Fraction
    FI_sum = FI
    FI_frac = np.empty((FI.shape[0],FI.shape[1]))
    for i in range(0,FI.shape[0]):
        for j in range(0,FI.shape[1]):
            if (FI[i,j]<=0):
                FI_sum[i,j]=0              # Let negative values zero first, and then calculate sum FI
        for j in range(0,FI_sum.shape[1]):
            if (np.sum(FI_sum[i,:])==0):
                FI_frac[i,j]=0
            else:
                FI_frac[i,j] = FI_sum[i,j]/np.sum(FI_sum[i,:])
                
    df_f =  pd.DataFrame(FI_frac) 
    

####### weighted

    df_perform = pd.read_csv(path+'/model_evaluation/Summary_performance_mean.csv')
    bacc = df_perform['Balanced Accuracy']

    weight = (bacc - 0.5)/0.5

    df_n_w = pd.DataFrame()
    df_f_w = pd.DataFrame()


    for i in range(0,df_n.shape[0]):
        df_n_w =  pd.concat([df_n_w,df_n.iloc[i,:] * weight[i]],axis=1)
        df_f_w =  pd.concat([df_f_w,df_f.iloc[i,:] * weight[i]],axis=1)


        

    df = pd.concat([df,pd.DataFrame(df.sum()).T],axis=0)
    df.columns=ROI_name
    df.index=algs
    
    df_n = pd.concat([df_n,pd.DataFrame(df_n.sum()).T],axis=0)
    df_n.columns=ROI_name
    df_n.index=algs
    
    df_f = pd.concat([df_f,pd.DataFrame(df_f.sum()).T],axis=0)    
    df_f.columns=ROI_name
    df_f.index=algs   
    
    
    df_n_w = pd.concat([df_n_w.T,pd.DataFrame(df_n_w.T.sum()).T],axis=0)
    df_f_w = pd.concat([df_f_w.T,pd.DataFrame(df_f_w.T.sum()).T],axis=0)

    
    df_n_w.columns=ROI_name
    df_n_w.index=algs
    df_f_w.columns=ROI_name
    df_f_w.index=algs
    
    return df,df_n,df_f,df_n_w,df_f_w

In [ ]:
refs=['Original','WC','CR'] #no reference, whole cerebellum, composite reference region
for r in refs:
    path = '~/results/AV45_cnad_'+r+'/'+'AV45_ctx_cnad_'+r+'bl' # STRAMLINE output folder ('experiment name/data name')
    print("ref: ",r)
    
    FI_p,ROI_name = readFI(path+'/model_evaluation/feature_importance/Permutation')

    # Rescaled Permutation FI
    exec('df_'+r+'_p, df_'+r+'_p_n, df_'+r+'_p_f, df_'+ r+'_p_n_w, df_'+r+'_p_f_w = rescaleFI(FI_p,ROI_name,path)')
    # FI name: e.g. df_CR_p_n_w (normalized weighted FI)
    
# Top20 regions
df_CR_p_n_w.T.sort_values(by="SUM",ascending=False).head(20)

# Heatmap (Fig.3)

In [ ]:
# Normalize
df_summary_n = pd.concat([df_Original_n,df_WC_n,df_CR_n],ignore_index = True)
# Fraction
df_summary_f = pd.concat([df_Original_f,df_WC_f,df_CR_f],ignore_index = True)

algs_Original = []
algs_WC = []
algs_CR = []
for i in range(0,algs.shape[0]):
    algs_Original.append( "_".join([algs[i],'Original']))
    algs_WC.append( "_".join([algs[i],'WC']))
    algs_CR.append( "_".join([algs[i],'CR']))
y_axis_labels = np.append(algs_Original,algs_WC)
y_axis_labels = np.append(y_axis_labels,algs_CR)
y_axis_labels

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme()

np.random.seed(42)

x_axis_labels = ROI_name
name = ["n","f"]
for i in range(0,2):
    plt.figure(figsize=(35,30))
    exec('ax = sns.heatmap(df_summary_'+name[i]+',xticklabels=x_axis_labels,yticklabels=y_axis_labels)')
    ax.set_title('Feature Importance, Cortex only_'+str(name[i]))
    ax.set(xlabel='Features', ylabel='Algorithms')
    save_fig("Normalied Feature Importance_ctx_"+str(name[i]))

# AD related region rank plot (Fig. 4)

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import cm

def rank_cal(df_n,df_f,df_n_w,df_f_w,FI_type,name):
    
    # Claculate the rank table for each algs and rescaling methods
    algs = np.array(['NB', 'EN', 'LR', 'DT', 'RF', 'GB', 'XGB', 'LGB', 'CGB', 'SVM', 'ANN', 'KNN', 'GP'])
    ROIs_target = np.array(['PRECUNEUS','ISTHMUSCINGULATE','POSTERIORCINGULATE','ANTERIORCINGULATE','MEDIALORBITOFRONTAL','LATERALORBITOFRONTAL', 'ROSTRALMIDDLEFRONTAL','SUPERIORFRONTAL', 'SUPERIORPARIETAL','SUPRAMARGINAL'])
   
    index_id = np.append(algs,np.array(['Mean_algs','All_n','All_n_w','All_f','All_f_w']))
    column_id = np.append(ROIs_target,np.array(['Sum']))
    rank = pd.DataFrame(index=index_id,columns=column_id) 


    for i in range(0,len(ROIs_target)):
        values = np.where(df_n.T.sort_values(by='SUM', ascending=False).index.str.contains(ROIs_target[i]))
        values_w = np.where(df_n_w.T.sort_values(by='SUM', ascending=False).index.str.contains(ROIs_target[i]))
        rank.iloc[-4,i] = values[0].mean()
        rank.iloc[-3,i] = values_w[0].mean()

        values = np.where(df_f.T.sort_values(by='SUM', ascending=False).index.str.contains(ROIs_target[i]))
        values_w = np.where(df_f_w.T.sort_values(by='SUM', ascending=False).index.str.contains(ROIs_target[i]))
        rank.iloc[-2,i] = values[0].mean()
        rank.iloc[-1,i] = values_w[0].mean()

        for j in range(0,df_n.shape[0]-1):
            values_s = np.where(df_n.T.sort_values(by=algs[j], ascending=False).index.str.contains(ROIs_target[i]))
            rank.iloc[j,i] = values_s[0].mean()
        rank.iloc[-5,i] = round(rank.iloc[:-5,i].mean(),1)
    rank["Sum"] = round(rank.sum(axis = 1),3)
    
    # Plot the bar plots
    plt.rcParams["figure.figsize"] = [12, 6]
    plt.style.use('tableau-colorblind10') 
    ax=rank.iloc[:,:rank.shape[1]-1].plot(kind="bar",stacked=True,cmap = cm.get_cmap('Spectral'),grid=False)#Set3
    plt.title(FI_type+" FI")
    plt.xlabel("Algorithms")
    plt.ylabel("FI ranking")
    legend = plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.3),
               ncol=5)
    legend.get_frame().set_alpha(None)
    legend.get_frame().set_facecolor((0, 0, 0, 0))

    for i in range(rank.shape[0]):
        p=ax.patches[i]
        width = p.get_width()
        ax.annotate('{:.2f}'.format(rank.iloc[i,-1]), (p.get_x()+width/2., (rank.iloc[i,-1])+12),ha='center',
                    va='center',fontsize=9, color='black')
    plt.ylim([0,300])
    plt.savefig(name, transparent=True,bbox_inches='tight')
    plt.close()

    return rank

In [ ]:
FI_type = 'Permutation'
refs=['WC','CR'] #whole cerebellum, composite reference region
for r in refs:
    name = '~/results/Rank_'+r+'_P.png'
    exec('rank_p_'+r+'=rank_cal(df_'+r+'_p_n, df_'+r+'_p_f, df_'+r+'_p_n_w,df_'+r+'_p_f_w,FI_type,name)')